# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [1]:
import sys
import random
import functools
from pprint import pprint, pformat
import numpy as np
from toolz import pipe

from leap_ec.individual import Individual
from leap_ec.ops import pool, cyclic_selection, clone

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation, add_segment, remove_segment, copy_segment

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import genome_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import genome_mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[array([1, 0, 1, 1, 0]), array([0, 0, 0, 0, 1]), array([0, 1, 1, 1, 1]), array([0, 0, 1, 0, 0])]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [array([0, 1, 0, 1, 1]), array([0, 0, 1, 1, 0]), array([0, 0, 0, 0, 1])]
1 [array([0, 0, 1, 1, 0]), array([0, 0, 0, 1, 0]), array([0, 0, 0, 0, 1]), array([0, 1, 0, 0, 0])]
2 [array([1, 1, 0, 0, 1])]
3 [array([0, 0, 0, 0, 1]), array([0, 1, 0, 1, 0]), array([1, 0, 1, 1, 0])]
4 [array([0, 1, 1, 0, 1]), array([1, 1, 0, 1, 0]), array([1, 1, 1, 1, 0]), array([0, 1, 0, 0, 0])]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [4]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [array([1, 3]), array([0, 6]), array([0, 1])]
1 [array([0, 6]), array([0, 2]), array([0, 1]), array([1, 0])]
2 [array([3, 1])]
3 [array([0, 1]), array([1, 2]), array([2, 6])]
4 [array([1, 5]), array([3, 2]), array([3, 6]), array([1, 0])]


In [5]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual(np.array([[0,0],[1,1]]))
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_bitflip))
print('mutated:', mutated)

original: [[0 0]
 [1 1]] None
mutated: [array([0, 1]), array([1, 1])] None


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [6]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pformat(s, indent=2))

0 [array([-3.40231119,  0.43604658,  5.08091891])]
1 [ array([ 1.19156977,  0.74468427, -9.3431648 ]),
  array([4.13376007, 0.67227527, 2.8389859 ]),
  array([-1.35036005, -0.16159234, -5.88014623])]
2 [ array([-3.61662821, -0.16085274,  4.82057054]),
  array([-2.55900795, -0.48237721, -5.81362419])]
3 [ array([-0.14272255,  0.21981472,  0.10823838]),
  array([-0.15293992, -0.821942  ,  1.62435248]),
  array([1.84334389, 0.64284763, 1.19385632])]
4 [array([ 1.39889923, -0.59903152,  6.37915447])]
5 [ array([-1.5994267 , -0.69050754,  2.0867158 ]),
  array([ 3.88378476,  0.87293692, -9.17930249]),
  array([-2.52557113, -0.39187786, -3.91009272]),
  array([4.05377848, 0.69027317, 4.70061672]),
  array([-3.01356327,  0.78726788, -3.22621981])]


Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [7]:
original = Individual(np.array([[0.0,0.0],[1.0,1.0],[-1.0,0.0]]))
print('original:', original)
mutated = next(apply_mutation(iter([original]), expected_num_mutations=3, mutator=genome_mutate_gaussian(std=1.0)))
print('mutated:', mutated)

original: [[ 0.  0.]
 [ 1.  1.]
 [-1.  0.]] None
mutated: [array([-0.94064896, -1.94892   ]), array([-0.24435059,  1.        ]), array([-1.97594838,  0.        ])] None


# Other pipeline operators

Besides the aformentioned `apply_mutation`, segmented representations have other pipeline operators, which are:

* `add_segment()`, possibly add a new segment
* `remove_segment()`, possibly remove a segment
* `copy_segment()`, possibly select and copy an existing segment


In [8]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               remove_segment(probability=1.0), 
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop))

pop: [Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
 Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
 Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
 Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
 Individual([[0, 0], [1, 1]], IdentityDecoder(), None)]
new_pop: [Individual([[1, 1]], IdentityDecoder(), None),
 Individual([[1, 1]], IdentityDecoder(), None),
 Individual([[1, 1]], IdentityDecoder(), None),
 Individual([[0, 0]], IdentityDecoder(), None),
 Individual([[1, 1]], IdentityDecoder(), None)]


In [9]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               copy_segment(probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None)]
new_pop: [        Individual([[1, 1], [0, 0], [1, 1]], IdentityDecoder(), None),
         Individual([[0, 0], [1, 1], [0, 0]], IdentityDecoder(), None),
         Individual([[0, 0], [0, 0], [1, 1]], IdentityDecoder(), None),
         Individual([[0, 0], [1, 1], [1, 1]], IdentityDecoder(), None),
         Individual([[0, 0], [1, 1], [1, 1]], IdentityDecoder(), None)]


In [10]:
# lastly, demonstrate add_segment, which generates an entirely new segment
test_sequence = [12345]  # just an arbitrary sequence for testing

def gen_sequence():
    """ return an arbitrary static test_sequence """
    return test_sequence

pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))

new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               add_segment(seq_initializer=gen_sequence, probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None),
     Individual([[0, 0], [1, 1]], IdentityDecoder(), None)]
new_pop: [        Individual([[0, 0], [12345], [1, 1]], IdentityDecoder(), None),
         Individual([[0, 0], [1, 1], [12345]], IdentityDecoder(), None),
         Individual([[0, 0], [1, 1], [12345]], IdentityDecoder(), None),
         Individual([[0, 0], [12345], [1, 1]], IdentityDecoder(), None),
         Individual([[0, 0], [1, 1], [12345]], IdentityDecoder(), None)]
